<a href="https://colab.research.google.com/github/onleey/Machine-Learning_demo/blob/master/651_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_%EC%9E%84%EB%B2%A0%EB%94%A9_%EB%A0%88%EC%8B%9C%ED%94%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/ai_chat_001'

Mounted at /content/drive
/content/drive/MyDrive/ai_chat_001


In [ ]:
import os
os.environ["JAVA_HOME"] = "/path/to/your/java"

In [ ]:
pip install --upgrade gensim konlpy

In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 43.2 MB/s eta 0:00:00


1_3 Word2Vec
- Word2Vec은 분산 표현 형태의 단어 임베딩 모델이다.
- Word2Vec은 2013년 구글에서 발표했으며 가장 많이 사용하고 있는 단어 임베딩 모델이다.
- Word2Vec 모델은 CBOW(continuous bag-of-words)와 skip-gram 두가지 모델이다.
- CBOW 모델 맥락(context word)이라 표현되는 주변 단어들을 이용해 타깃단어를 예측하는 신경망 모델이다. 신경망의 입력을 주변 단어들로 구성하고 출력을 타깃 단어로 설정해 학습된 가중치 데이터를 임베딩 벡터로 활용한다.
- skp-gram모델은 CBOW 모델과 반대로 하나의 타깃 단어를 이용해 주변 단어들을 예측하는 신경망 모델이다. 입출력이 CBOW모델과 반대로 되어 있기 때문에 skip-gram모델이 CBOW 모델에 비해 예측해야 하는 맥락이 많아진다. 따라서, 단어 분산 표현력이 우수해 CBOW 모델에 비해 임베딩 품질이 우수하다. 반면 CBOW모델은 타깃 단어의 손실만 계산하면 되기 때문에 학습 속도가 빠른 장점이 있다.
- CBOW 모델에서는 타깃 단어를 예측하기 위해 앞뒤 단어를 확인할 수 있다. 이때 앞뒤로 몇개의 단어까지 확인할지 결정할 수 있는데 이 범위를 윈도우(window)라고한다. 예를 들어 윈도우 크기를 2로 한 경우 타깃 단어의 앞뒤 단어 2개까지 모델 학습을 위해 사용된다.
- Word2Vec의 단어 임베딩은 해당 단어를 밀집 벡터로 표현하며 학습을 통해 의미상 비슷한 단어들을 비슷한 벡터 공간에 위치시킨다. 또한 벡터 특성상 의미에 따라 방향을 갖게 된다.
- 임베딩된 벡터들 간 연산이 가능하기 때문에 단어 간 관계를 계산할 수 있다. '왕'과'여왕'의 방향 차이만큼 '남자'와'여자'의 방향 차이가 생긴다.

In [ ]:
import os
from konlpy.tag import Komoran

# Java 환경 변수 설정
os.environ["JAVA_HOME"] = "/path/to/your/java"

# Komoran 객체 초기화
komoran = Komoran()


In [ ]:
from konlpy.tag import Kkma

kkma = Kkma()
docs = [kkma.nouns(sentence[0]) if sentence and sentence[0] else [] for sentence in recipe_data]


Word2Vec

In [ ]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
from konlpy.tag import Kkma
import time


# 레시피 데이터 읽어옴
def read_recipe_data(recipe):
    with open(recipe, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] # header 제거
    return data


# 측정 시작
start = time.time()

# 리뷰 파일 읽어오기
print('1) 말뭉치 데이터 읽기 시작')
recipe_data = read_recipe_data('./recipe_data/chicken.txt')
print(len(recipe_data)) # 리뷰 데이터 전체 개수
print('1) 말뭉치 데이터 읽기 완료: ', time.time() - start)

# 문장단위로 명사만 추출해 학습 입력 데이터로 만듬
print('2) 형태소에서 명사만 추출 시작')
komoran = Komoran()
kkma = Kkma()
docs = [kkma.nouns(sentence[0]) if sentence and sentence[0] else [] for sentence in recipe_data]


print('2) 형태소에서 명사만 추출 완료: ', time.time() - start)
print(docs[:10])

# word2vec 모델 학습
print('3) word2vec 모델 학습 시작')
# vector_size: 단어 임베딩 벡터의 차원(크기), window: 주변 단어 윈도우의 크기
# hs:1(모델학습에 softmax 사용), 0(negative 옵션 값이 0이 아닐때 음수 샘플링으로 사용)
# min_count: 단어 최소 빈도 수 제한(설정된 min_count  빈도 수 이하의 단어들은 학습하지 않음)
# sg: 0(CBOW 모델), 1(skip-gram 모델)
model = Word2Vec(sentences=docs, vector_size=200, window=4, min_count=2, sg=1)
print('3) word2vec 모델 학습 완료: ', time.time() - start)

# 모델 저장
print('4) 학습된 모델 저장 시작')
model.save('./recipe_data/nvmc.model')
print('4) 학습된 모델 저장 완료: ', time.time() - start)

# 학습된 말뭉치 개수, 코퍼스 내 전체 단어 개수
print("corpus_count : ", model.corpus_count)
print("corpus_total_words : ", model.corpus_total_words)



1) 말뭉치 데이터 읽기 시작
29170
1) 말뭉치 데이터 읽기 완료:  0.02897167205810547
2) 형태소에서 명사만 추출 시작
2) 형태소에서 명사만 추출 완료:  377.55873703956604
[['조', '조회수', '회수', '24,000'], ['기준', '3', '분', '조리', '조리시간', '시간', '60', '60분', '이내', '난이도', '초급'], [], [], [], ['닭', '닭안심살500', '안심살', '500'], ['우유', '우유1.5컵', '1.5', '컵'], ['양파', '양파중간1개', '중간', '1', '개'], ['생강술', '생강술1', '1', '술'], ['간장', '간장1.5', '1.5', '술']]
3) word2vec 모델 학습 시작
3) word2vec 모델 학습 완료:  379.43989396095276
4) 학습된 모델 저장 시작
4) 학습된 모델 저장 완료:  379.4920508861542
corpus_count :  29170
corpus_total_words :  126517


In [ ]:
print(dir(model))
#print(model.sorted_vocab)
print(dir(model.wv))
print(model.wv.key_to_index)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__ignoreds', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__numpys', '__recursive_saveloads', '__reduce__', '__reduce_ex__', '__repr__', '__scipys', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_adapt_by_suffix', '_check_corpus_sanity', '_check_training_sanity', '_clear_post_train', '_do_train_epoch', '_do_train_job', '_get_next_alpha', '_get_thread_working_mem', '_job_producer', '_load_specials', '_log_epoch_end', '_log_epoch_progress', '_log_progress', '_log_train_end', '_raw_word_count', '_save_specials', '_scan_vocab', '_smart_save', '_train_epoch', '_train_epoch_corpusfile', '_worker_loop', '_worker_loop_corpusfile', 'add_lifecycle_event', 'add_null_word', 'alpha', 'batch_words', 'build_vocab', 'build_vocab_from_freq', 'cbow_mean', 'comment', 'compute_l

word2vec 모델활용
위에서 생성된 word2vec 모델 파일을 읽어와 실제로 단어 임베딩된 값과 벡터 공간상의 유사한

In [ ]:
from gensim.models import Word2Vec

# 모델 로딩
model = Word2Vec.load('./recipe_data/nvmc.model')
print("corpus_total_words : ", model.corpus_total_words)

# '사랑'이란 단어로 생성한 단어 임베딩 벡터
print(len(model.wv['레시피']))
print('레시피 : ', model.wv['레시피'])

# 단어 유사도 계산
# model.wv.similarity( ) : 두 단어 간의 유사도를 계산
print("육류 = 소고기\t", model.wv.similarity(w1='돼지고기', w2='소고기'))
print("채소 = 배추\t", model.wv.similarity(w1='버섯', w2='배추'))
print("곡류 = 쌀\t", model.wv.similarity(w1='콩', w2='쌀'))
print("유제품!= 우유\t", model.wv.similarity(w1='치즈', w2='우유'))
print("가공식품 != 스팸\t", model.wv.similarity(w1='소세지', w2='스팸'))





corpus_total_words :  126517
200
레시피 :  [-0.0629123  -0.12909204  0.07071701  0.25110412  0.0939095  -0.17946611
 -0.00848219  0.08795907 -0.0444739   0.05019715 -0.11028331 -0.11432531
 -0.05279882  0.05504731 -0.20177536 -0.21065715 -0.26717734  0.02338087
  0.0852813  -0.0661082   0.03900494 -0.16079964  0.00875382 -0.01775199
  0.09588443 -0.02149993 -0.04194206 -0.13016932 -0.16537945 -0.0294292
  0.18806587  0.08699249 -0.1474475   0.28809962  0.11446823  0.13051933
  0.06340865 -0.00750034  0.05506762 -0.3638423  -0.13967788 -0.04685608
 -0.02145632  0.07355327  0.13935502 -0.14746018 -0.0517886   0.02759809
  0.0210413   0.0397637   0.12922046 -0.05570972  0.03719113 -0.11409853
 -0.08192063  0.023587    0.02198736 -0.16447482 -0.0974237  -0.02600023
  0.06519882  0.08751576 -0.06409791 -0.02388195 -0.18697351  0.08485965
 -0.04226204  0.29325795 -0.16520196  0.34579185 -0.01184064  0.0293976
  0.0761761   0.11259311 -0.01453219 -0.07807665  0.06637057 -0.21892402
 -0.15176171 

In [ ]:
# 가장 유사한 단어 추출
# model.wv.most_similar( ) : 인자로 사용한 단어와 가장 유사한 단어를 리스트로 반환한다.
# 즉, 벡터 공간에서 가장 가까운 거리에 있는 단어들을 반환한다. topn인자는 반환되는 유사한 단어 수를 의미하며,
# 예제에서는 5개까지 유사한 단어를 반환한다.

print(model.wv.most_similar("육류", topn=5))
print(model.wv.most_similar("소고기", topn=5))
print(model.wv.most_similar(["육류","소고기"], topn=5))

# 안성기와 최고와 밀접한 증거들 중에  조재현이 방해가 되니까 조재현 제외

print(model.wv.most_similar(["육류","소고기"],negative=['양파'], topn=5))


[('파송', 0.9908769726753235), ('건더기파', 0.9905950427055359), ('소형', 0.990481972694397), ('가쓰', 0.9896628856658936), ('보기', 0.9895431995391846)]
[('껍질채', 0.9935906529426575), ('채수가', 0.9926372766494751), ('정사각형', 0.9922573566436768), ('입자', 0.9918208718299866), ('세로', 0.9910677075386047)]
[('해놓읍', 0.9950251579284668), ('이해', 0.994581937789917), ('해놓', 0.9944067001342773), ('이정', 0.9941051602363586), ('보세', 0.9937824606895447)]
[('간식만들기', 0.9538052082061768), ('로스트치킨', 0.9532651901245117), ('반절', 0.9524943828582764), ('팝콘', 0.9517480134963989), ('현미밥1공기', 0.9512350559234619)]
